In [5]:
import moyra as ma
import moyra.elements as ele
import moyra.forces as ef
import sympy as sym
import sympy.physics.mechanics as me
import warnings
import matplotlib.cbook
import numpy as np

warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
me.mechanics_printing()

#### Create Symbols

In [6]:
DoFs = 2          # total degrees of freedom
main_panels = 10  # main wing panels
htp_panels = 4    # horizontal tailplane panels

p = ma.DynamicModelParameters(DoFs)
p.L_ot = ma.ModelSymbol(string = 'L_ot')
p.L_tc = ma.ModelSymbol(string = 'L_tc')
p.L_tw = ma.ModelSymbol(string = 'L_tw')
p.L_te = ma.ModelSymbol(string = 'L_te')

p.m = ma.ModelSymbol(string = 'm')
p.I_xx = ma.ModelSymbol(string = 'I_xx')
p.I_yy = ma.ModelSymbol(string = 'I_yy')
p.I_zz = ma.ModelSymbol(string = 'I_zz')

p.y_i = sym.Symbol('y_i') # spanwise location

## Numeric Model Constants
p.a = ma.ModelMatrix(value =[np.pi*2]*main_panels,string='a',length=main_panels) # local 2d lift curve slope at each panel on main wing
p.a_h = ma.ModelMatrix(value =[np.pi*2]*htp_panels,string='a_h',length=htp_panels) # local 2d lift curve slope at each panel on horizontal tailplane
p.a_0 = ma.ModelSymbol(value = 2*np.pi, string = 'a_0')   # placeholder for local c_l


p.c = ma.ModelSymbol(value = 0.15,string = 'c') # chord of wing
p.c_h = ma.ModelSymbol(value = 0.15,string = 'c_htp') # chord of htp

p.b = ma.ModelSymbol(value = 0.15,string = 'b') # chord of wing
p.b_h = ma.ModelSymbol(value = 0.15,string = 'b_htp') # chord of htp

# Attitude Parmas
p.rho = ma.ModelSymbol(value = 1.225,string = 'rho')                 # density
p.V = ma.ModelSymbol(value = 10,string = 'V')                        # velocity
p.g  = ma.ModelSymbol(value = 9.81,string = 'g')                     # gravity


#### Create Coordinate Systems

In [7]:
tether_frame = ma.HomogenousTransform().R_z(p.q[0]).R_x(p.q[1]).Translate(0,0,-p.L_ot)

#Generate Mass Matrices
M = ele.MassMatrix(p.m, I_xx = p.I_xx, I_yy=p.I_yy, I_zz=p.I_zz)
M = ele.MassMatrix(p.m, I_xx = 0, I_yy=0, I_zz=0)


# Generate Rigid Elements
aircraft_mass = ele.RigidElement(tether_frame.Translate(p.L_tc,0,0),M,gravityPotential=True)

# Main Wing Aero Forces 
wing_AeroForces = ef.AeroForce.PerUnitSpan(p,tether_frame.Translate(p.L_tw,p.y_i,0),p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = 0,
                               rootAlpha = 0,
                               alpha_zero = 0,
                               stall_angle = 0,
                               c = p.c,
                               c_d_max = 0,
                               linear = True,
                                z_inverted=False)

# HTP Aero Forces 
htp_AeroForces = ef.AeroForce.PerUnitSpan(p,tether_frame.Translate(p.L_te,p.y_i,0),p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = 0,
                               rootAlpha = 0,
                               alpha_zero = 0,
                               stall_angle = 0,
                               c = p.c_h,
                               c_d_max = 0,
                               linear = True,
                                z_inverted=False)


In [8]:
# split wing into segments
forces = []
for i in range(main_panels):
    seg_width = p.b/main_panels
    yi = seg_width/2 + i*seg_width - p.b/2
    forces.append(wing_AeroForces.subs({p.y_i:yi,p.a_0:p.a[main_panels-(i+1)]})*seg_width)
for i in range(htp_panels):
    seg_width = p.b_h/htp_panels
    yi = seg_width/2 + i*seg_width - p.b_h/2
    forces.append(wing_AeroForces.subs({p.y_i:yi,p.a_0:p.a_h[htp_panels-(i+1)]})*seg_width)
Q = sym.Matrix([0]*p.qs)
for f in forces:
    Q += f.Q()
AeroForces = ef.ExternalForce(Q)

In [9]:
## print the velocity in the wing frame
tether_frame = ma.HomogenousTransform().R_z(p.q[0]).R_x(p.q[1]).Translate(0,0,-p.L_ot)
T = tether_frame.Translate(p.L_tw,p.y_i,0)
BJ = sym.trigsimp(sym.powsimp(sym.cancel(sym.expand(T.BodyJacobian(p.q)))))
sym.simplify(BJ*p.qd)

⎡-(Lₒₜ⋅sin(q₁) + yᵢ⋅cos(q₁))⋅q₀̇⎤
⎢                              ⎥
⎢   Lₒₜ⋅q₁̇ + L_tw⋅cos(q₁)⋅q₀̇   ⎥
⎢                              ⎥
⎢   -L_tw⋅sin(q₁)⋅q₀̇ + yᵢ⋅q₁̇   ⎥
⎢                              ⎥
⎢              q₁̇              ⎥
⎢                              ⎥
⎢          sin(q₁)⋅q₀̇          ⎥
⎢                              ⎥
⎣          cos(q₁)⋅q₀̇          ⎦

In [10]:
sm = ma.SymbolicModel.FromElementsAndForces(p,[aircraft_mass],ExtForces=AeroForces)

In [11]:
sm.M

⎡   2      2           2                      ⎤
⎢Lₒₜ ⋅m⋅sin (q₁) + L_tc ⋅m  Lₒₜ⋅L_tc⋅m⋅cos(q₁)⎥
⎢                                             ⎥
⎢                                    2        ⎥
⎣   Lₒₜ⋅L_tc⋅m⋅cos(q₁)            Lₒₜ ⋅m      ⎦

In [12]:
sym.simplify(sm.f)

⎡Lₒₜ⋅m⋅(2⋅Lₒₜ⋅cos(q₁)⋅q₀̇ - L_tc⋅q₁̇)⋅sin(q₁)⋅q₁̇⎤
⎢                                             ⎥
⎢          ⎛                2    ⎞            ⎥
⎣    Lₒₜ⋅m⋅⎝- Lₒₜ⋅cos(q₁)⋅q₀̇  + g⎠⋅sin(q₁)    ⎦

In [13]:
sm.ExtForces.Q()

⎡                ⎛               9⋅b⋅cos(q₁)⎞ ⎛                   9⋅b⋅q₁̇⎞    
⎢  a₀⋅L_tw⋅b⋅c⋅ρ⋅⎜-Lₒₜ⋅sin(q₁) - ───────────⎟⋅⎜-L_tw⋅sin(q₁)⋅q₀̇ + ──────⎟⋅sin
⎢                ⎝                    20    ⎠ ⎝                     20  ⎠     
⎢- ───────────────────────────────────────────────────────────────────────────
⎢                                          20                                 
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎣                                                                             

                        ⎛               7⋅b⋅cos(q₁)⎞ ⎛                   7⋅b⋅q
(q₁)⋅q₀̇   a₁⋅L_tw⋅b⋅c⋅ρ⋅⎜-Lₒₜ⋅sin(q₁) - ──────────

In [10]:
sm.to_matlab_file(p,'')

In [14]:
nm = ma.NumericModel.from_SymbolicModel(p,sm)